In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MISTRAL_KEY= os.getenv("MISTRAL_KEY")
GEMINI_APIL_KEY= os.getenv("GEMINI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

In [ ]:
import pathlib

# defining paths to output and data directories to be used throughout notebook
BASE_DIR = pathlib.Path.cwd().resolve().parents[1]

DATA_DIR = BASE_DIR /"data"
OUT_DIR = BASE_DIR / "inference" / "outputs"

## Preparing MMLU Dataset for Inference

In [ ]:
from datasets import load_dataset
import pandas as pd

# loading dataset from hugging face
dataset = load_dataset("TIGER-Lab/MMLU-Pro", split="test[:1000]")
df = dataset.to_pandas()

df = df[["question_id", "question", "options", "answer"]].rename(columns={"question_id": "id", "options": "choices"})
df.to_csv(DATA_DIR / "mmlu_pro_1k.csv", index=False)

print("Cleaned and saved:", len(df), "rows →", DATA_DIR / "mmlu_pro_1k.csv")
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

calib_df, eval_df = train_test_split(df, test_size=0.85, random_state=42, shuffle=True)
calib_df.to_csv(DATA_DIR / "calibration_split.csv", index=False)
eval_df.to_csv(DATA_DIR / "evaluation_split.csv", index=False)

print("Split complete:")
print("Calibration:", len(calib_df), "rows")
print("Evaluation :", len(eval_df), "rows")

In [ ]:
print("Calibration sample:")
display(calib_df.head(3))

print("Evaluation sample:")
display(eval_df.head(3))

---

## QWEN

Loading model to local device

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import gc

major, minor = map(int, transformers.__version__.split('.')[:2])
if (major, minor) < (4, 37):
    raise ValueError(f"Transformers {transformers.__version__} too old.")

MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"
print(f"Loading: {MODEL_NAME}")

if torch.backends.mps.is_available():
    device = torch.device("mps")
    dtype = torch.float16
else:
    device = torch.device("cpu")
    dtype = torch.float32

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=dtype,
    device_map=None, 
    token=HF_TOKEN
).to(device)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Model loaded successfully on {device} with dtype={dtype}")


Running inference

In [ ]:

import pandas as pd
import re
import time
from collections import Counter
import torch
import gc

THRESHOLDS = [0.25, 0.5, 0.75, 0.9]

# helper to extract model's answer
# If model says "I don't know" -> IDK
# otherwise pull first letter
def extract_letter(text):
    text_clean = text.strip().upper()

    if "I DON'T KNOW" in text_clean:
        return "IDK"

    m = re.search(r"\b([A-D])\b", text_clean)
    return m.group(1) if m else None

# helper function to reate the multiple choice prompt for model
# dynamically inserts t
# formats the choices as A/B/C/D options
def build_mcq_prompt(row, t):
    opts = row["choices"]
    # if choices column is a string  "['A', 'B', 'C']" convert it back to list
    if isinstance(opts, str):
        try:
            opts = ast.literal_eval(opts)
        except Exception:
            pass

    lines = [f"Q: {row['question']}", "Options:"]
    if isinstance(opts, (list, tuple)):
        labels = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        for i, choice in enumerate(opts):
            lines.append(f"{labels[i]}. {choice}")
    else:
        lines.append(str(opts))
    
    # prompt
    # confidence instructions
    # answer ONLY if its confidence > t
    # otherwise explicitly say "I don't know".
    lines.append(
        f"\nAnswer only if you are more than {t} confident. "
        f"Mistakes incur {t}/(1−{t}) points, correct = +1, IDK = 0. "
        "If unsure, respond exactly with 'I don't know.'\n"
        "Provide only the capital letter of your chosen answer (A, B, C, D, ...) "
        "or 'I don't know'."
    )
    return "\n".join(lines)

# generates n samples using temperature sampling
# treating each generation as vote for an answer.
# final prediction = majority vote.
# p = (# votes for majority) / n.
def qwen_answer_and_conf(prompt, n=6, temperature=0.7):
    votes = []
    for _ in range(n):
        try:
            messages = [{"role": "user", "content": prompt}]
            formatted_prompt = tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )

            inputs = tokenizer(
                formatted_prompt,
                return_tensors="pt",
                truncation=True,
                max_length=512
            ).to(model.device)

            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=10,
                    temperature=temperature,
                    do_sample=True if temperature > 0 else False,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id
                )

            txt = tokenizer.decode(
                outputs[0][inputs['input_ids'].shape[1]:],
                skip_special_tokens=True
            ).strip()

            letter = extract_letter(txt)
            if letter:
                votes.append(letter)

            del inputs, outputs
            torch.cuda.empty_cache()

        except Exception as e:
            print("Error:", e)

        time.sleep(0.1)

    if not votes:
        return "IDK", 0.0

    counts = Counter(votes)
    pred = counts.most_common(1)[0][0]
    conf = counts[pred] / len(votes)
    return pred, float(conf)


subset_df = pd.read_csv(DATA_DIR/"evaluation_split.csv")

qwen_rows = []
total_q = len(subset_df)

# sample n times
# convert each output to letter / IDK
# majority vote for prediction
# p = freq(pred) / n
for t in THRESHOLDS:
    print(f"\nRunning inference for threshold t={t}")
    for i, row in subset_df.iterrows():
        # building prompt with curr t
        prompt = build_mcq_prompt(row, t=t) 
        
        print(f"[t={t} | {i+1}/{total_q}] Asking Qwen2.5-1.5B ...")
        pred, conf = qwen_answer_and_conf(prompt, n=6)
        
        qwen_rows.append({
            "id": row["id"],
            "threshold": t,
            "question": row["question"],
            "choices": row["choices"],
            "answer": row["answer"],
            "predicted_answer": pred,
            "confidence": conf
        })

        if (i + 1) % 5 == 0:
            gc.collect()
            torch.cuda.empty_cache()

qwen_out = pd.DataFrame(qwen_rows)
qwen_path= OUT_DIR / f"qwen-mmlu.csv"
qwen_out.to_csv(qwen_path, index=False)
print(f"Qwen2.5-1.5B predictions for all thresholds saved to {qwen_path}")
qwen_out.head(5)

---

## GPT-4o-mini

In [ ]:
from openai import OpenAI
import pandas as pd
import numpy as np
import re, time, ast, os
from collections import Counter

client = OpenAI(api_key=OPENAI_API_KEY)

THRESHOLDS = [0.25, 0.50, 0.75, 0.90]

MODEL_NAME = "gpt-4o-mini"
DATASET_NAME = "mmlu"

OUTPUT_PATH = OUT_DIR / f"gpt-{DATASET_NAME}.csv"
print("Saving to:", OUTPUT_PATH)

eval_df = pd.read_csv(DATA_DIR / "evaluation_split.csv")
print("Loaded evaluation_split.csv", len(eval_df), "rows")

# helper function to reate the multiple choice prompt for model
# dynamically inserts t
# formats the choices as A/B/C/D options
def build_mcq_prompt(row, t):
    opts = row["choices"]
    # if choices column is a string  "['A', 'B', 'C']" convert it back to list
    if isinstance(opts, str):
        try:
            opts = ast.literal_eval(opts)
        except Exception:
            pass
    labels = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    lines = [f"Q: {row['question']}", "Options:"]
    # this is what we want
    if isinstance(opts, (list, tuple)):
        for i, choice in enumerate(opts):
            lines.append(f"{labels[i]}. {choice}")
    else:
        lines.append(str(opts))
        
    # prompt
    # confidence instructions
    # answer ONLY if its confidence > t
    # otherwis explicitly say "I don't know"
    lines.append(
        f"\nAnswer only if you are more than {t} confident. "
        f"Mistakes incur {t}/(1−{t}) penalty points, correct = +1, IDK = 0. "
        "If unsure, respond exactly with 'I don't know'.\n"
        "Provide ONLY the capital letter (A, B, C, …) OR 'I don't know'."
    )

    return "\n".join(lines)

# helper to extract model's answer
# If model says "I don't know" -> IDK
# otherwise pull first letter
def extract_letter(text):
    clean = text.strip().upper()
    if "I DON'T KNOW" in clean:
        return "IDK"
    match = re.search(r"\b([A-Z])\b", clean)
    return match.group(1) if match else None

# generates n samples using temperature sampling
# treating each generation as vote for an answer
# final prediction = majority vote
# p = (# votes for majority) / n
def gpt_answer_and_conf(prompt, n=6, temperature=0.7, sleep_s=0.4):
    votes = []
    for _ in range(n):
        try:
            resp = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=temperature,
                max_tokens=10
            )
            txt = resp.choices[0].message.content.strip()
            letter = extract_letter(txt)
            if letter:
                votes.append(letter)
        except Exception as e:
            print("GPT Error:", e)

        time.sleep(sleep_s)

    if not votes:
        return "IDK", 0.0

    counts = Counter(votes)
    pred = counts.most_common(1)[0][0]
    conf = counts[pred] / len(votes)
    return pred, float(conf)


if OUTPUT_PATH.exists():
    existing = pd.read_csv(OUTPUT_PATH)
    saved_rows = existing.to_dict("records")
    done_pairs = set(zip(existing["id"], existing["threshold"]))
    print("Resuming — loaded", len(saved_rows), "rows.")
else:
    saved_rows = []
    done_pairs = set()
    print("Starting fresh.")

SAVE_EVERY = 10
new_since_save = 0
total = len(eval_df)


# sample n times
# convert each output to letter / IDK
# majority vote for prediction
# p = freq(pred) / n
for t in THRESHOLDS:
    print(f"\n Threshold t={t}")

    for idx, row in eval_df.iterrows():
        key = (row["id"], float(t))
        if key in done_pairs:
            continue

        prompt = build_mcq_prompt(row, t)
        print(f"[t={t}] {idx+1}/{total} → GPT...")

        pred, conf = gpt_answer_and_conf(prompt)

        saved_rows.append({
            "id": row["id"],
            "threshold": t,
            "question": row["question"],
            "choices": row["choices"],
            "answer": row["answer"],
            "predicted_answer": pred,
            "confidence": conf
        })
        done_pairs.add(key)
        new_since_save += 1

        if new_since_save >= SAVE_EVERY:
            pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
            print(f"Saved {len(saved_rows)} rows.")
            new_since_save = 0


pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
print("saved:", OUTPUT_PATH)

---

## Gemini

In [ ]:
import google.generativeai as genai
import pandas as pd
import numpy as np
import re, time, ast, os
from collections import Counter

genai.configure(api_key=GEMINI_API_KEY)

THRESHOLDS = [0.25, 0.50, 0.75, 0.90]

MODEL_NAME = "gemini-2.5-flash-lite"
DATASET_NAME = "mmlu"

OUTPUT_PATH = OUT_DIR / f"{MODEL_NAME}-{DATASET_NAME}.csv"
print("Saving to:", OUTPUT_PATH)

eval_df = pd.read_csv(DATA_DIR / "evaluation_split.csv")
print("Loaded evaluation_split.csv", len(eval_df), "rows")


# helper function to reate the multiple choice prompt for model
# dynamically inserts t
# formats the choices as A/B/C/D options
def build_mcq_prompt(row, t):
    opts = row["choices"]
    # if choices column is a string  "['A', 'B', 'C']" convert it back to list
    if isinstance(opts, str):
        try:
            opts = ast.literal_eval(opts)
        except:
            pass

    labels = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    lines = [f"Q: {row['question']}", "Options:"]

    if isinstance(opts, (list, tuple)):
        for i, choice in enumerate(opts):
            lines.append(f"{labels[i]}. {choice}")
    else:
        lines.append(str(opts))

    # prompt
    # confidence instructions
    # answer ONLY if its confidence > t
    # otherwis explicitly say "I don't know"
    lines.append(
        f"\nAnswer only if you are more than {t} confident. "
        f"Mistakes incur {t}/(1−{t}) penalty points, correct = +1, IDK = 0. "
        "If unsure, respond exactly with 'I don't know'.\n"
        "Provide ONLY the capital letter (A, B, C, …) OR 'I don't know'."
    )

    return "\n".join(lines)

# helper function for gemini reponse
def extract_gemini_text(resp):
    try:
        return resp.text
    except:
        pass

    try:
        return resp.candidates[0].content.parts[0].text
    except:
        return ""

# helper to extract model's answer
# If model says "I don't know" -> IDK
# otherwise pull first letter
def extract_letter(text):
    clean = text.strip().upper()
    if "I DON'T KNOW" in clean:
        return "IDK"
    m = re.search(r"\b([A-Z])\b", clean)
    return m.group(1) if m else None

gemini_model = genai.GenerativeModel(MODEL_NAME)

# generates n samples using temperature sampling
# treating each generation as vote for an answer
# final prediction = majority vote
# p = (# votes for majority) / n
def gemini_answer_and_conf(prompt, n=6, temperature=0.7, base_sleep=1.5):
    votes = []
    for _ in range(n):
        while True:
            try:
                resp = gemini_model.generate_content(
                    prompt,
                    generation_config=genai.types.GenerationConfig(
                        temperature=temperature,
                        max_output_tokens=10
                    )
                )
                txt = extract_gemini_text(resp).strip()
                letter = extract_letter(txt)
                if letter:
                    votes.append(letter)
                time.sleep(base_sleep)
                break
            # auto retrying on 429 rate limits
            # sleeps between calls to refresh limit
            except Exception as e:
                err = str(e)
                print("Gemini Error:", err)
                if "429" in err or "Quota exceeded" in err:
                    retry_sec = 45
                    m = re.search(r"retry_delay.*?(\d+)", err)
                    if m:
                        retry_sec = int(m.group(1))
                    print(f"Rate limit hit. Sleeping {retry_sec} seconds…")
                    time.sleep(retry_sec)
                    continue
                print("Non-quota error. Sleeping 5 seconds…")
                time.sleep(5)
                continue
    if not votes:
        return "IDK", 0.0
    counts = Counter(votes)
    pred = counts.most_common(1)[0][0]
    conf = counts[pred] / len(votes)
    return pred, float(conf)

if OUTPUT_PATH.exists():
    existing = pd.read_csv(OUTPUT_PATH)
    saved_rows = existing.to_dict("records")
    done_pairs = set(zip(existing["id"], existing["threshold"]))
    print("Resuming — loaded", len(saved_rows), "rows.")
else:
    saved_rows = []
    done_pairs = set()
    print("Starting fresh.")

SAVE_EVERY = 10
new_since_save = 0
total = len(eval_df)

# sample n times
# convert each output to letter / IDK
# majority vote for prediction
# p = freq(pred) / n
for t in THRESHOLDS:
    print(f"\nThreshold t={t}")
    for idx, row in eval_df.iterrows():
        key = (row["id"], float(t))
        if key in done_pairs:
            continue
        prompt = build_mcq_prompt(row, t)
        print(f"[t={t}] {idx+1}/{total} → Gemini...")
        pred, conf = gemini_answer_and_conf(prompt)
        saved_rows.append({
            "id": row["id"],
            "threshold": t,
            "question": row["question"],
            "choices": row["choices"],
            "answer": row["answer"],
            "predicted_answer": pred,
            "confidence": conf
        })
        done_pairs.add(key)
        new_since_save += 1
        if new_since_save >= SAVE_EVERY:
            pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
            print(f"Saved {len(saved_rows)} rows.")
            new_since_save = 0
pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
print("saved:", OUTPUT_PATH)

---

---

## Mistral

In [ ]:
import mistralai
import pandas as pd
import numpy as np
import re, time, ast, os
from collections import Counter


client = mistralai.Mistral(api_key=MISTRAL_KEY)

MODEL_NAME = "open-mistral-7b"
DATASET_NAME = "mmlu"

OUTPUT_PATH = OUT_DIR / f"{MODEL_NAME}-{DATASET_NAME}.csv"
print("Saving to:", OUTPUT_PATH)

eval_df = pd.read_csv(DATA_DIR / "evaluation_split.csv")
print("Loaded evaluation_split.csv →", len(eval_df), "rows")

# helper function to reate the multiple choice prompt for model
# dynamically inserts t
# formats the choices as A/B/C/D options
def build_mcq_prompt(row, t):
    opts = row["choices"]
    # if choices column is a string  "['A', 'B', 'C']" convert it back to list
    if isinstance(opts, str):
        try:
            opts = ast.literal_eval(opts)
        except:
            pass

    labels = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    lines = [f"Q: {row['question']}", "Options:"]
    # this is what we want
    if isinstance(opts, (list, tuple)):
        for i, c in enumerate(opts):
            lines.append(f"{labels[i]}. {c}")
    else:
        lines.append(str(opts))
    
    # prompt
    # confidence instructions
    # answer ONLY if its confidence > t
    # otherwis explicitly say "I don't know"
    lines.append(
        f"\nAnswer only if you are more than {t} confident. "
        f"Mistakes incur {t}/(1−{t}) penalty points. "
        "Correct = +1, IDK = 0. "
        "If unsure, respond exactly with 'I don't know'.\n"
        "Provide ONLY the capital letter (A, B, C, …) OR 'I don't know'."
    )
    return "\n".join(lines)

# helper to extract model's answer
# If model says "I don't know" -> IDK
# otherwise pull first letter
def extract_letter(text):
    clean = text.strip().upper()

    if "I DON'T KNOW" in clean or clean == "IDK" or "IDK" in clean:
        return "IDK"

    m = re.search(r"\b([A-Z])\b", clean)
    return m.group(1) if m else None

# generates n samples using temperature sampling
# treating each generation as vote for an answer
# final prediction = majority vote
# p = (# votes for majority) / n
def mistral_answer_and_conf(prompt, n=6, temperature=0.7, base_sleep=1.2):
    votes = []

    for _ in range(n):
        while True:
            try:
                resp = client.chat.complete(
                    model=MODEL_NAME,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=temperature,
                    max_tokens=10,
                )

                txt = resp.choices[0].message.content.strip()
                letter = extract_letter(txt)

                if letter:
                    votes.append(letter)

                time.sleep(base_sleep)
                break

            except Exception as e:
                err = str(e)
                print("Mistral Error:", err)

                if "429" in err or "rate" in err.lower():
                    print("Rate limit hit — sleeping 30 sec.")
                    time.sleep(30)
                    continue

                print("Unexpected error — sleeping 5 sec.")
                time.sleep(5)
                continue

    if not votes:
        return "IDK", 0.0

    counts = Counter(votes)
    pred = counts.most_common(1)[0][0]
    conf = counts[pred] / len(votes)
    return pred, conf

if OUTPUT_PATH.exists():
    existing = pd.read_csv(OUTPUT_PATH)
    saved_rows = existing.to_dict("records")
    done_pairs = set(zip(existing["id"], existing["threshold"]))
    print("Resuming — loaded", len(saved_rows), "rows.")
else:
    saved_rows = []
    done_pairs = set()
    print("Starting fresh.")

SAVE_EVERY = 10
new_since_save = 0
total = len(eval_df)

THRESHOLDS = [0.25, 0.50, 0.75, 0.90]

# sample n times
# convert each output to letter / IDK
# majority vote for prediction
# p = freq(pred) / n
for t in THRESHOLDS:
    print(f"\n=== Threshold t={t} ===")

    for idx, row in eval_df.iterrows():
        key = (row["id"], float(t))
        if key in done_pairs:
            continue

        prompt = build_mcq_prompt(row, t)
        print(f"[t={t}] {idx+1}/{total} → Mistral API...")

        pred, conf = mistral_answer_and_conf(prompt)

        saved_rows.append({
            "id": row["id"],
            "threshold": t,
            "question": row["question"],
            "choices": row["choices"],
            "answer": row["answer"],
            "predicted_answer": pred,
            "confidence": conf
        })

        done_pairs.add(key)
        new_since_save += 1

        if new_since_save >= SAVE_EVERY:
            pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
            print(f"Saved {len(saved_rows)} rows.")
            new_since_save = 0

pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
print("saved:", OUTPUT_PATH)

---

## llama

Loading model to local device

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch, os
import torch.nn.functional as F

# logged in and downloaded model from hugging face

MODEL_NAME = "meta-llama/Llama-3.2-3B"
DTYPE = torch.bfloat16
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu"

# loading tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.2-3B",
    token=os.getenv("HF_TOKEN"),
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B",
    token=os.getenv("HF_TOKEN"),
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

model.eval()
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu"

Llama Inference

In [ ]:
THRESHOLDS = [0.25, 0.50, 0.75, 0.90]
DATASET_NAME = "mmlu"
OUTPUT_PATH = OUT_DIR /  OUT_DIR / f"{MODEL_NAME}-{DATASET_NAME}.csv"

print("Saving to:", OUTPUT_PATH)

eval_df = pd.read_csv(DATA_DIR / "evaluation_split.csv")
print("Loaded evaluation_split.csv", len(eval_df), "rows")

# helper function to reate the multiple choice prompt for model
# dynamically inserts t
# formats the choices as A/B/C/D options
def build_mcq_prompt(row, t):
    opts = row["choices"]
    # if choices column is a string  "['A', 'B', 'C']" convert it back to list
    if isinstance(opts, str):
        try:
            opts = ast.literal_eval(opts)
        except:
            pass
    labels = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    lines = [f"Q: {row['question']}", "Options:"]
    # this is right
    if isinstance(opts, (list, tuple)):
        for i, choice in enumerate(opts):
            lines.append(f"{labels[i]}. {choice}")
    else:
        lines.append(str(opts))
    # prompt
    # confidence instructions
    # answer ONLY if its confidence > t
    # otherwis explicitly say "I don't know"
    lines.append(
        f"\nAnswer only if you are more than {t} confident. "
        f"Mistakes incur {t}/(1−{t}) penalty points, correct = +1, IDK = 0. "
        "If unsure, respond exactly with 'I don't know'.\n"
        "Provide ONLY the capital letter (A, B, C, …) OR 'I don't know'."
    )
    return "\n".join(lines)

# helper to extract model's answer
# If model says "I don't know" -> IDK
# otherwise pull first letter
def extract_letter(text):
    clean = text.strip().upper()
    if "I DON'T KNOW" in clean:
        return "IDK"
    m = re.search(r"\b([A-Z])\b", clean)
    return m.group(1) if m else None

# generates n samples using temperature sampling
# treating each generation as vote for an answer
# final prediction = majority vote
# p = (# votes for majority) / n
def llama_answer_and_conf(prompt, n=6, temperature=0.7, top_p=0.9, sleep_s=0.1):
    votes = []

    for _ in range(n):
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=2048
        ).to(DEVICE)

        with torch.no_grad():
            out = model.generate(
                **inputs,
                max_new_tokens=12,
                do_sample=True,
                temperature=temperature,
                top_p=top_p,
                pad_token_id=tokenizer.eos_token_id
            )

        gen_ids = out[0][inputs["input_ids"].shape[1]:]
        txt = tokenizer.decode(
            gen_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        )

        letter = extract_letter(txt)
        if letter:
            votes.append(letter)

        time.sleep(sleep_s)

    if not votes:
        return "IDK", 0.0

    counts = Counter(votes)
    pred = counts.most_common(1)[0][0]
    conf = counts[pred] / len(votes)
    return pred, float(conf)

saved_rows = []
done_pairs = set()
print("Starting fresh from scratch.")


SAVE_EVERY = 10
new_since_save = 0
total = len(eval_df)


# sample n times
# convert each output to letter / IDK
# majority vote for prediction
# p = freq(pred) / n
for t in THRESHOLDS:
    print(f"\n=== Threshold t={t} ===")

    for idx, row in eval_df.iterrows():
        key = (row["id"], float(t))
        if key in done_pairs:
            continue

        print(f"[t={t}] {idx+1}/{total} → Llama...")

        prompt = build_mcq_prompt(row, t)
        pred, conf = llama_answer_and_conf(prompt)

        saved_rows.append({
            "id": row["id"],
            "threshold": t,
            "question": row["question"],
            "choices": row["choices"],
            "answer": row["answer"],
            "predicted_answer": pred,
            "confidence": conf
        })

        done_pairs.add(key)
        new_since_save += 1

        if new_since_save >= SAVE_EVERY:
            pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
            print(f"Saved {len(saved_rows)} rows.")
            new_since_save = 0

# Final save
pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
print("Saved:", OUTPUT_PATH)

---

## Claude

In [ ]:
from anthropic import Anthropic
import pandas as pd
import numpy as np
import re, time, ast, os
from collections import Counter

client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

THRESHOLDS = [0.25, 0.50, 0.75, 0.90]

MODEL_NAME = "claude"
DATASET_NAME = "mmlu"

OUTPUT_PATH = OUT_DIR / f"{MODEL_NAME}-{DATASET_NAME}.csv"
print("Saving to:", OUTPUT_PATH)

eval_df = pd.read_csv(DATA_DIR / "evaluation_split.csv")
print("Loaded evaluation_split.csv ", len(eval_df), "rows")

# helper function to reate the multiple choice prompt for model
# dynamically inserts t
# formats the choices as A/B/C/D options
def build_mcq_prompt(row, t):
    opts = row["choices"]
    # if choices column is a string  "['A', 'B', 'C']" convert it back to list
    if isinstance(opts, str):
        try:
            opts = ast.literal_eval(opts)
        except:
            pass

    labels = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    lines = [f"Q: {row['question']}", "Options:"]
    # this is how we want it
    if isinstance(opts, (list, tuple)):
        for i, choice in enumerate(opts):
            lines.append(f"{labels[i]}. {choice}")
    else:
        lines.append(str(opts))
    # prompt
    # confidence instructions
    # answer ONLY if its confidence > t
    # otherwis explicitly say "I don't know"
    lines.append(
        f"\nAnswer only if you are more than {t} confident. "
        f"Mistakes incur {t}/(1−{t}) penalty points, correct = +1, IDK = 0. "
        "If unsure, respond exactly with 'I don't know'.\n"
        "Provide ONLY the capital letter (A, B, C, …) OR 'I don't know'."
    )

    return "\n".join(lines)

# claude specific helper to parse its answer
def extract_claude_text(resp):
    try:
        return resp.content[0].text
    except:
        return ""
    
# helper to extract model's answer
# If model says "I don't know" -> IDK
# otherwise pull first letter
def extract_letter(text):
    clean = text.strip().upper()
    if "I DON'T KNOW" in clean:
        return "IDK"
    m = re.search(r"\b([A-Z])\b", clean)
    return m.group(1) if m else None

# generates n samples using temperature sampling
# treating each generation as vote for an answer
# final prediction = majority vote
# p = (# votes for majority) / n
def claude_answer_and_conf(prompt, n=6, temperature=0.7, base_sleep=1.2):
    votes = []

    for _ in range(n):
        while True:
            try:
                resp = client.messages.create(
                    model="claude-haiku-4-5",
                    max_tokens=20,
                    temperature=temperature,
                    messages=[
                        {"role": "user", "content": prompt}
                    ]
                )

                txt = extract_claude_text(resp).strip()
                letter = extract_letter(txt)
                if letter:
                    votes.append(letter)

                time.sleep(base_sleep)
                break

            except Exception as e:
                err = str(e)
                print("Claude Error:", err)

                # sleeping in case rate limit is hit
                if "rate_limit" in err or "429" in err:
                    print("🔁 Rate limit — sleeping 30s…")
                    time.sleep(30)
                    continue
                print("⚠️ Non-rate error — sleeping 5s…")
                time.sleep(5)
                continue

    if not votes:
        return "IDK", 0.0

    counts = Counter(votes)
    pred = counts.most_common(1)[0][0]
    conf = counts[pred] / len(votes)
    return pred, float(conf)

if OUTPUT_PATH.exists():
    existing = pd.read_csv(OUTPUT_PATH)
    saved_rows = existing.to_dict("records")
    done_pairs = set(zip(existing["id"], existing["threshold"]))
    print("Resuming — loaded", len(saved_rows), "rows.")
else:
    saved_rows = []
    done_pairs = set()
    print("Starting fresh.")

SAVE_EVERY = 10
new_since_save = 0
total = len(eval_df)


# sample n times
# convert each output to letter / IDK
# majority vote for prediction
# p = freq(pred) / n
for t in THRESHOLDS:
    print(f"\n=== Threshold t={t} ===")

    for idx, row in eval_df.iterrows():
        key = (row["id"], float(t))
        if key in done_pairs:
            continue

        prompt = build_mcq_prompt(row, t)

        print(f"[t={t}] {idx+1}/{total} → Claude...")

        pred, conf = claude_answer_and_conf(prompt)

        saved_rows.append({
            "id": row["id"],
            "threshold": t,
            "question": row["question"],
            "choices": row["choices"],
            "answer": row["answer"],
            "predicted_answer": pred,
            "confidence": conf
        })

        done_pairs.add(key)
        new_since_save += 1

        if new_since_save >= SAVE_EVERY:
            pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
            print(f"Saved {len(saved_rows)} rows.")
            new_since_save = 0

pd.DataFrame(saved_rows).to_csv(OUTPUT_PATH, index=False)
print("Saved:", OUTPUT_PATH)